In [1]:
import pandas as pd 
import numpy as np  
from pyfile.connect2db import * # 데이터베이스 연결
conn,cursor = create_con()
# DB에 있는 테이블 목록 확인하기
pd.read_sql('select tbl_name from sqlite_master;',conn)

,tbl_name
0,cafe_post_list_naver_test
1,cafe_post_comment_naver
2,cafe_post_contents_naver
3,cafe_post_list_naver
4,comments_hy_kona_naver
5,commnets_kia_stonic_naver
6,commnets_sam_qm3_naver
7,commnets_ssy_tivoli_naver
8,kona
9,kona_total


# 자연어 처리 

In [2]:
import numpy as np
import pandas as pd
import re 
import time

from collections import Counter
from konlpy.tag import Okt, Kkma, Mecab, Hannanum, Komoran
t = Okt()
kkma = Kkma()
# mecab = Mecab() # Mecab은 윈도우에서 사용불가
hannanum = Hannanum()
komoran = Komoran()

# Mecab은 윈도우에서 사용불가하기 때문에 eunjeon라이브러리를 사용
from eunjeon import Mecab
mecab = Mecab()

## 게시글 데이터 교정 후 텍스트 마이닝

In [3]:
df_contents = pd.read_sql('select * from cafe_post_contents_naver;',conn,parse_dates=['td_date'])
df_posts = pd.read_sql('select * from cafe_post_list_naver;',conn,parse_dates=['p_date'])

df2 = pd.merge(df_contents[['car_name','inner_number','td_date','tbody']],
         df_posts[['car_name','article_number','article','article_cmt','p_view','location2']],
         left_on = ['car_name','inner_number'],right_on=['car_name','article_number'])
df2 = df2[['car_name','td_date','location2','tbody','p_view','article_cmt']].copy()
df2.head(3)

,car_name,td_date,location2,tbody,p_view,article_cmt
0,스토닉,2018-02-12,인천,\n TUIX에서 벌써 코나는 바디킷이나왔네요빨리스토닉도나오면좋겠군요 \n,403,1
1,스토닉,2018-02-07,인천,\n브렌톤으로 주문하러고하는데요 기존기으 떼기어려운가요??\n,425,3
2,스토닉,2018-02-04,인천,\n내일 아는동생이랑 순정휠 도색하려고합니다.혹시하신분들 후회하시나요?만족하시나요?...,309,5


In [4]:
from pyfile.correct_sentence import * 

In [5]:
# 20개만 테스트 해보는 것으로 설정 
after_correct = correct_sentence(df2)

16.607168912887573 초


In [6]:
after_correct = pd.Series(after_correct).apply(lambda x : re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\\"\'|\(\)\[\]\<\>`\'…》\xa0]|ㅋ|ㅎ|ㅠ|ㅜ', '',x))
joined_contents = ' '.join(after_correct)
words = mecab.nouns(joined_contents)

In [7]:
words[:10]

['코나', '바디', '킷', '토닉', '브렌', '톤', '주문', '존기', '가요', '동생']

## 분리된 단어를 26가지 기준으로 재분류 

In [8]:
words = pd.read_csv('./data/게시글_뉴스_단어분류결과.csv',engine='python',encoding='utf-8')[['단어','분류']]
words.groupby('분류').\
agg({'단어':'count'}).\
sort_values('단어',ascending=False).\
head(8)

,단어
분류,
가격/구입조건,104
품질,77
제조회사/브랜드,66
첨단/편의사양,64
외관스타일,63
실내인테리어,60
기본사양,55
신모델/최신모델,50


# 데이터 준비
## 카페 게시글 작성자에게 성별과 연령대 특징을 랜덤으로 부과
**성별과 연령대를 부여하는 방법은 신차등록 대수의 지역별, 성별, 연령별 비중을 기준으로 함**

In [9]:
from pyfile.data_frame_for_ml import * 
car_name = 'QM3'
model = car_names.loc[car_name,'en']

cafe = create_cafe_member(model,car_name,conn,cursor)
cafe.head(5)

,car_name,article_number,p_date,tbody,p_view,등록지역,성별,연령대,특징
0,QM3,482758,2017-07-15,\n진흙탕에 빠졌습니다.. 암담했습니다...평택에 친구만나러 가고나서 만나서 애기하...,443,경남,여자,50대,38
1,QM3,515212,2018-10-08,\n- 모델명:qm3- 연식 :14- 키로수 :4만8천- 추가옵션 :- 질문내용 :...,323,인천,여자,50대,164
2,QM3,514454,2018-09-25,\n시동걸고 기어 p(파킹)에 두고 엑셀페달을 계속밟아도 차에 아무문제없나요?기어 ...,233,서울,여자,50대,122
3,QM3,514187,2018-09-19,\n- 모델명:qm3- 연식 :15- 키로수 :4.7- 추가옵션 :- 질문내용 :부...,153,경기,여자,40대,23
4,QM3,513331,2018-09-03,\n - 모델명:qm3- 연식 :15- 키로수 :89000- 추가옵션 :- qm3 ...,345,경기,여자,40대,23


## 각 모델별로 5개의 가상의 카페를 생성 

In [10]:
for car_name in ['QM3']:#'스토닉','티볼리','코나']:
    for i in range(5):
        model = car_names.loc[car_name,'en']

        cafe = create_cafe_member(model,car_name,conn,cursor)
        cafe.to_excel('./data/cafes/{0}_카페_{1}.xlsx'.format(car_name,str(i+1)),index=False,encoding='utf-8')
        print(car_name,i+1,'번째 카페 데이터 생성 완료')

QM3 1 번째 카페 데이터 생성 완료
QM3 2 번째 카페 데이터 생성 완료
QM3 3 번째 카페 데이터 생성 완료
QM3 4 번째 카페 데이터 생성 완료
QM3 5 번째 카페 데이터 생성 완료


# Kmeans 비지도 학습 알고리즘 적용

In [11]:
from pyfile.Kmeans_ml import *  

In [12]:
model = 'QM3'
df_result = kmeans_ml(model)

In [13]:
df_result.iloc[0:5,0:7]

,등록지역,성별,연령대,가격/구입조건,실내인테리어,크기(전장),승차감
0,서울,남자,70대,적음,적음,적음,적음
1,강원,남자,50대,적음,적음,적음,적음
2,경기,여자,20대,적음,적음,적음,적음
3,경기,여자,40대,보통,보통,보통,보통
4,경기,여자,30대,보통,보통,보통,보통


## 4개 모델에 대해 머신러닝 결과 통합 

In [14]:
df = pd.read_excel('./data/머신러닝_결과_통합표.xlsx')
df.iloc[0:5,0:7]

,등록지역,성별,연령대,가격/구입조건,실내인테리어,크기(전장),승차감
0,서울,여자,30대,많음,많음,많음,많음
1,제주,남자,40대,적음,적음,적음,적음
2,대구,남자,30대,적음,적음,적음,적음
3,경기,여자,20대,많음,많음,많음,많음
4,경기,남자,40대,많음,많음,많음,많음


### 베뉴 카페 게시글 300여 개를 수집해 텍스트 마이닝 후 통계

In [15]:
venu = pd.read_excel('./data/베뉴_게시글_단어_집계.xlsx')
venu.sort_values('단어',ascending=False).head(5).\
style.set_caption('베뉴 카페 게시글에서 많이 언급된 단어들')

,분류,단어
12,차의 컬러,9
13,첨단/편의사양,9
1,가격/구입조건,8
2,기본사양,7
3,승차감,5


In [16]:
category = '첨단/편의사양'
df2 = pd.melt(df,id_vars=['등록지역','성별','연령대']) 
df2['counts'] = 1 
df3 = df2.query('variable == "{}"'.format(category)).\
groupby(['성별','연령대','value']).agg({'counts':np.sum}).\
pivot_table(index=['성별','연령대'],columns='value')['counts']
total = df3.sum(axis=1)
df4 =df3.copy() 
for col in df3.columns:
    df4[col] = df3[col]/total
df4.fillna(0).style.format('{:.2%}').\
background_gradient(cmap='bwr',low=2,high=1).\
set_caption('{}에 대한 관심'.format(category))